In [1]:
%load_ext sql
import pandas as pd
import sqlite3

In [2]:
%sql sqlite:///CourseData.db

'Connected: @CourseData.db'

In [3]:
%%sql

DROP TABLE IF EXISTS COURSE_CATALOG;
CREATE TABLE COURSE_CATALOG (
    cat_id INTEGER NOT NULL PRIMARY KEY,
    catalog_id TEXT,
    catalog_year INTEGER,
    program_id INTEGER NOT NULL,
    course_title TEXT NOT NULL,
    credits INTEGER NOT NULL,
    prereqs TEXT,
    coreqs TEXT,
    fees TEXT,
    attributes TEXT,
    description TEXT,
FOREIGN KEY (program_id) REFERENCES PROGRAMS(program_id));
DROP TABLE IF EXISTS PROGRAMS;
CREATE TABLE PROGRAMS (
    program_id INTEGER NOT NULL PRIMARY KEY,
    program_code TEXT NOT NULL, 
    program_name TEXT NOT NULL);
/* ***    FOREIGN KEY (program_code) REFERENCES COURSE_CATALOG(program_code));*** */
DROP TABLE IF EXISTS COURSES;
CREATE TABLE COURSES (
    course_id INTEGER NOT NULL PRIMARY KEY,
    crn TEXT,
    catalog_id TEXT,
    term_id TEXT,
    section INTEGER,
    professor_id TEXT,
    cap INTEGER,
    act INTEGER,
    rem INTEGER,
    FOREIGN KEY (catalog_id) REFERENCES COURSE_CATALOG(catalog_id));
DROP TABLE IF EXISTS COURSE_MEETINGS;
CREATE TABLE COURSE_MEETINGS (
    crn INTEGER NOT NULL PRIMARY KEY,
    location TEXT NOT NULL,
    day TEXT NOT NULL,
    start TEXT NOT NULL,
    end TEXT NOT NULL,
    FOREIGN KEY (crn) REFERENCES COURSES (crn));
DROP TABLE IF EXISTS PROFESSORS;
CREATE TABLE PROFESSORS (
    professor_id INTEGER NOT NULL PRIMARY KEY,
    name TEXT);
DROP TABLE IF EXISTS TERM_CATALOG;
CREATE TABLE TERM_CATALOG (
    term_id INTEGER NOT NULL PRIMARY KEY,
    term TEXT,
    catalog_year TEXT);
    

 * sqlite:///CourseData.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

Used the following protocol to load the source data into tables:
Open a new Git Terminal
> sqlite3 CourseData.db
> .mode csv
> .import CourseCatalog2017_2018.csv Import_CourseCatalog
> .import CourseCatalog2018_2019.csv Import_CourseCatalog
> .import course_meetings_fall2014.csv Import_course_meetings
> .import course_meetings_fall2015.csv Import_course_meetings
> .import course_meetings_fall2016.csv Import_course_meetings
> .import course_meetings_fall2017.csv Import_course_meetings
> .import course_meetings_fall2018.csv Import_course_meetings
> .import course_meetings_spring2015.csv Import_course_meetings
> .import course_meetings_spring2016.csv Import_course_meetings
> .import course_meetings_spring2017.csv Import_course_meetings
> .import course_meetings_spring2018.csv Import_course_meetings
> .import course_meetings_spring2019.csv Import_course_meetings
> .import course_meetings_springbreak2017.csv Import_course_meetings
> .import course_meetings_summer2015.csv Import_course_meetings
> .import course_meetings_summer2016.csv Import_course_meetings
> .import course_meetings_summer2017.csv Import_course_meetings
> .import course_meetings_summer2018.csv Import_course_meetings
> .import course_meetings_winter2015.csv Import_course_meetings
> .import course_meetings_winter2016.csv Import_course_meetings
> .import course_meetings_winter2017.csv Import_course_meetings
> .import course_meetings_winter2018.csv Import_course_meetings
> 
> .import courses_fall2014.csv Import_courses
> .import courses_fall2015.csv Import_courses
> .import courses_fall2016.csv Import_courses
> .import courses_fall2017.csv Import_courses
> .import courses_fall2018.csv Import_courses
> .import courses_spring2015.csv Import_courses
> .import courses_spring2016.csv Import_courses
> .import courses_spring2017.csv Import_courses
> .import courses_spring2018.csv Import_courses
> .import courses_spring2019.csv Import_courses
> .import courses_springbreak2017.csv Import_courses
> .import courses_summer2015.csv Import_courses
> .import courses_summer2016.csv Import_courses
> .import courses_summer2017.csv Import_courses
> .import courses_summer2018.csv Import_courses
> .import courses_winter2015.csv Import_courses
> .import courses_winter2016.csv Import_courses
> .import courses_winter2017.csv Import_courses
> .import courses_winter2018.csv Import_courses

In [4]:
%%sql
DROP TABLE IF EXISTS Import_Courses2;
DROP TABLE IF EXISTS Import_CourseCatalog2;
DROP TABLE IF EXISTS Import_Course_Meetings2;

 * sqlite:///CourseData.db
Done.
Done.
Done.


[]

In [5]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///CourseData.db', echo=False)

In [6]:
calendaryears = ['2017_2018','2018_2019']

for calendaryear in calendaryears:
    filepath = 'SourceData/Catalogs/CourseCatalog'+calendaryear+'.csv'
    data=pd.read_csv(filepath)
    data['calendaryear'] = calendaryear
    data.to_sql('Import_CourseCatalog2',con=engine,if_exists='append',index=False)

In [7]:
#it was pointed out in class that the above can be done programatically rather than manually

#for the courses and course meeting imports:



terms = ['Fall2014','Fall2015','Fall2016','Fall2017','Fall2018',
         'Spring2015','Spring2016','Spring2017','Spring2018','Spring2019','SpringBreak2017',
         'Summer2015','Summer2016','Summer2017','Summer2018',
          'Winter2015','Winter2016','Winter2017','Winter2018']

for term in terms:
    filepath = 'SourceData/'+term+'/courses.csv'
    data=pd.read_csv(filepath)
    data.to_sql('Import_Courses2',con=engine,if_exists='append',index=False)

    filepath = 'SourceData/'+term+'/course_meetings.csv'
    data=pd.read_csv(filepath)
    data.to_sql('Import_Course_Meetings2',con=engine,if_exists='append',index=False)

In [8]:
%%sql
/* *** data validation for CourseCatalog2 Imports*** */
SELECT
    (SELECT COUNT(*) FROM Import_CourseCatalog2) AS 'COUNTALL',
    (SELECT COUNT(*) FROM (SELECT DISTINCT * FROM Import_CourseCatalog2)) AS 'DISTINCT';

 * sqlite:///CourseData.db
Done.


COUNTALL,DISTINCT
4440,4440


In [9]:
%%sql
/* *** data validation for Courses2 Imports*** */
SELECT
    (SELECT COUNT(*) FROM Import_Courses2) AS 'COUNTALL',
    (SELECT COUNT(*) FROM (SELECT DISTINCT * FROM Import_Courses2)) AS 'DISTINCT';

 * sqlite:///CourseData.db
Done.


COUNTALL,DISTINCT
15937,15937


In [10]:
%%sql
/* *** data validation for Course_Meetings2 Imports*** */
SELECT
    (SELECT COUNT(*) FROM Import_Course_Meetings2) AS 'COUNTALL',
    (SELECT COUNT(*) FROM (SELECT DISTINCT * FROM Import_Course_Meetings2)) AS 'DISTINCT';

 * sqlite:///CourseData.db
Done.


COUNTALL,DISTINCT
284907,284847


%%sql
/* *** This entire cell is markdown as obsolete but kept for future reference*** */
/* *** THIS CODE WAS CREATED TO CLEAN UP THE IMPORT TABLE.  IT'S SUSPECTED THAT IMPORT_COURSES HAD DUPLICATE CRN, CAUSING INTEGRITY ISSUES*** */
DROP TABLE IF EXISTS Import_courses2;
CREATE TABLE Import_courses2 (
    crn TEXT,
    term TEXT,
    catalog_id TEXT,
    section TEXT,
    credits TEXT,
    title TEXT,
    meetings TEXT,
    timecodes TEXT,
    primary_instructor TEXT,
    cap INTEGER,
    act INTEGER,
    rem INTEGER);
INSERT INTO Import_courses2 (crn,term,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem)
    SELECT DISTINCT crn, term,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem FROM Import_courses;

In [11]:
%%sql
/* *** data test *** */
SELECT * FROM Import_CourseCatalog LIMIT 2;



 * sqlite:///CourseData.db
Done.


program_code,program_name,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description
AN,Asian Studies,AN 0301,Independent Study,1-3 Credits,,,,,Students undertake an individualized program of study in consultation with a director from the Asian studies faculty.
AN,Asian Studies,AN 0310,Asian Studies Seminar,3 Credits,,,,,"This seminar examines selected topics concerning Asia. This course is taught in conjunction with another 100-300 level course from a rotation of course offerings. Consult the Asian Studies director to identify the conjoined course for a given semester. The seminar concentrates on topics within the parameters of the conjoined course syllabus but adds research emphasis. Students registered for this course must complete a research project, to include 300-level research, in addition to the regular research requirements of the conjoined course, and a 25-50 page term paper in substitution of some portion of the conjoined course requirements, as determined by the instructor. Open to juniors and seniors only."


In [12]:
%%sql
/* *** data test*** */
SELECT * FROM Import_Courses
ORDER BY crn LIMIT 2;


 * sqlite:///CourseData.db
Done.


term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
Winter2015,10893,BI 0075,01,3.000,Ecology and Society,[],[],Thomas B. Cunningham,20,22,-2
Winter2015,10894,CO 0100,01,3.000,Human Communication Theories,[],[],David Gudelunas,20,14,6


In [13]:
%%sql
DELETE FROM PROGRAMS;
INSERT INTO PROGRAMS (program_code, program_name)
    SELECT DISTINCT program_code, program_name FROM Import_CourseCatalog
    ORDER BY program_code;


 * sqlite:///CourseData.db
0 rows affected.
84 rows affected.


[]

In [14]:
%%sql
DELETE FROM PROFESSORS;
INSERT INTO PROFESSORS (name)
    SELECT DISTINCT primary_instructor
    FROM Import_Courses2
    WHERE primary_instructor <> 'TBA' AND primary_instructor NOT LIKE '%/%';

 * sqlite:///CourseData.db
0 rows affected.
1095 rows affected.


[]

In [15]:
%%sql
SELECT * FROM TERM_CATALOG LIMIT 2;

 * sqlite:///CourseData.db
Done.


term_id,term,catalog_year


In [16]:
%%sql
DELETE FROM COURSE_CATALOG;
INSERT INTO COURSE_CATALOG (catalog_year,program_id,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description)
    SELECT DISTINCT C.calendaryear, P.program_id, C.catalog_id, C.course_title,C.credits,C.prereqs,C.coreqs,C.fees,C.attributes,C.description 
    FROM Import_CourseCatalog2 AS C
    JOIN PROGRAMS AS P
        USING(program_code);




 * sqlite:///CourseData.db
0 rows affected.
4440 rows affected.


[]

In [17]:
%%sql
DELETE FROM TERM_CATALOG;
INSERT INTO TERM_CATALOG (term,catalog_year)
    SELECT DISTINCT term, C2.catalog_year
    FROM Import_courses2
    JOIN COURSE_CATALOG AS C2 USING (catalog_id)
    
    ORDER BY term;

 * sqlite:///CourseData.db
0 rows affected.
38 rows affected.


[]

In [18]:
%%sql
SELECT * FROM COURSE_CATALOG LIMIT 2;

 * sqlite:///CourseData.db
Done.


cat_id,catalog_id,catalog_year,program_id,course_title,credits,prereqs,coreqs,fees,attributes,description
1,AN 0301,2017_2018,4,Independent Study,1-3 Credits,None,None,None,None,Students undertake an individualized program of study in consultation with a director from the Asian studies faculty.
2,AN 0310,2017_2018,4,Asian Studies Seminar,3 Credits,None,None,None,None,"This seminar examines selected topics concerning Asia. This course is taught in conjunction with another 100-300 level course from a rotation of course offerings. Consult the Asian Studies director to identify the conjoined course for a given semester. The seminar concentrates on topics within the parameters of the conjoined course syllabus but adds research emphasis. Students registered for this course must complete a research project, to include 300-level research, in addition to the regular research requirements of the conjoined course, and a 25-50 page term paper in substitution of some portion of the conjoined course requirements, as determined by the instructor. Open to juniors and seniors only."


In [19]:
%%sql
DELETE FROM COURSES;
INSERT INTO COURSES (crn,catalog_id,term_id,section,professor_id,cap,act,rem)
    SELECT DISTINCT C1.crn, COURSE_CATALOG.catalog_id,TERM_CATALOG.term_id,C1.section,C1.primary_instructor,C1.cap,C1.act,C1.rem
    FROM  Import_Courses2 AS C1
    LEFT JOIN PROFESSORS 
    ON C1.primary_instructor=PROFESSORS.name
    LEFT JOIN TERM_CATALOG USING (term)
    LEFT JOIN COURSE_CATALOG
    ON C1.catalog_id = COURSE_CATALOG.catalog_id AND COURSE_CATALOG.catalog_year=TERM_CATALOG.catalog_year
    ORDER BY crn;

 * sqlite:///CourseData.db
0 rows affected.
31874 rows affected.


[]

In [20]:
%%sql
DELETE FROM COURSE_MEETINGS;
INSERT INTO COURSE_MEETINGS (crn,location,day,start,end)
    SELECT DISTINCT COURSES.crn, C2.location,C2.day,C2.start,C2.end
    FROM COURSES
    JOIN Import_course_meetings AS C2
    ON 

 * sqlite:///CourseData.db
0 rows affected.
(sqlite3.OperationalError) incomplete input
[SQL: INSERT INTO COURSE_MEETINGS (crn,location,day,start,end)
    SELECT DISTINCT COURSES.crn, C2.location,C2.day,C2.start,C2.end
    FROM COURSES
    JOIN Import_course_meetings AS C2
    ON]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [21]:
%%sql 
SELECT * FROM Import_Courses2 
ORDER BY 
primary_instructor
LIMIT 10;

 * sqlite:///CourseData.db
Done.


term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
Spring2015,32262,IL 0299,A,3.000,Independent Study,"[{'days': 'TBA', 'times': '0', 'dates': 'TBA', 'location': ""IDEA Course Eval - NO and Int'l Studies Applied Course and Professional Studies BS and Prof Studies: Professions""}]","[""TBA 0 TBA IDEA Course Eval - NO and Int'l Studies Applied Course and Professional Studies BS and Prof Studies: Professions""]",01/20-04/30,0,0,0
Spring2015,32930,CO 0570,01,3.000,Project Proposal,"[{'days': 'TBA', 'times': '0', 'dates': 'TBA', 'location': 'IDEA Course Eval - NO and Professional Studies BA and Prof Studies: Humanities and Prof Studies: Social Sciences'}]",['TBA 0 TBA IDEA Course Eval - NO and Professional Studies BA and Prof Studies: Humanities and Prof Studies: Social Sciences'],01/20-05/08,0,0,0
Spring2015,32527,CO 0571,01,3.000,Independent Project,"[{'days': 'TBA', 'times': '0', 'dates': 'TBA', 'location': 'IDEA Course Eval - NO'}]",['TBA 0 TBA IDEA Course Eval - NO'],01/20-05/08,0,0,0
Spring2015,34111,CO 0572,01,3.000,Continuing Project,"[{'days': 'TBA', 'times': '0', 'dates': 'TBA', 'location': 'IDEA Course Eval - NO'}]",['TBA 0 TBA IDEA Course Eval - NO'],01/20-05/08,0,0,0
Spring2015,32941,CO 0572,02,3.000,Continuing Project,"[{'days': 'TBA', 'times': '0', 'dates': 'TBA', 'location': 'IDEA Course Eval - NO'}]",['TBA 0 TBA IDEA Course Eval - NO'],01/20-05/08,0,0,0
Summer2018,58467,NS 0687F,01,1.000,DNP Immersion,"[{'days': 'TBA', 'times': '0', 'dates': 'TBA', 'location': 'IDEA Course Eval - NO'}]",['TBA 0 TBA IDEA Course Eval - NO'],05/21-08/17,1,-1,0
Fall2014,73278,IN 0214,01,3.000,Commercial Design,"[{'days': 'TBA', 'times': '10', 'dates': 'TBA', 'location': 'IDEA Course Eval - YES'}]",['TBA 10 TBA IDEA Course Eval - YES'],08/15-01/15,0,10,0
Fall2015,75192,MU 0088,B,1.000,Private Lessons: Violin,"[{'days': 'TBA', 'times': '0', 'dates': 'TBA', 'location': 'IDEA Course Eval - NO'}]",['TBA 0 TBA IDEA Course Eval - NO'],09/01-12/07,0,0,0
Fall2014,73239,QA 0400,02,3.000,Applied Business Statistics,"[{'days': 'TBA', 'times': '0', 'dates': 'TBA', 'location': 'IDEA Course Eval - YES'}]",['TBA 0 TBA IDEA Course Eval - YES'],09/02-12/19,0,0,999
Fall2014,73284,CN 0465,02,0.000,Introduction to Substance Abuse and Addictions,"[{'days': 'TBA', 'times': '1', 'dates': 'TBA', 'location': 'IDEA Course Eval - YES'}]",['TBA 1 TBA IDEA Course Eval - YES'],09/02-12/30,0,1,0


## Test the integrity of your new `CourseData.db` database
- Each column has a sensible data type (Domain integrity); are there truncation or translation errors?   
- Each row describes a unique entity (Entity integrity); just having a PK is not enough: you will need to look for duplicate a data records
- Each relationship is implemented correctly (Relational integrity); are the FKs JOIN-compatible with the PKs? does each mandatory relationship have a corresponding NOT NULL constraint?
- Annotate your queries in Markdown cells so we know what you are testing and why.

In [22]:
%%sql 
SELECT DISTINCT PROFESSORS.name, SUM(act) AS SumStudents
FROM PROFESSORS
JOIN COURSE_CATALOG
ON name = primary_instructor;

 * sqlite:///CourseData.db
(sqlite3.OperationalError) no such column: act
[SQL: SELECT DISTINCT PROFESSORS.name, SUM(act) AS SumStudents
FROM PROFESSORS
JOIN COURSE_CATALOG
ON name = primary_instructor;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [23]:
%%sql
DELETE FROM COURSES;
INSERT INTO COURSES (crn,catalog_id,term_id,section,professor_id,cap,act,rem)
    SELECT DISTINCT C1.crn, COURSE_CATALOG.catalog_id,TERM_CATALOG.term_id,C1.section,C1.primary_instructor,C1.cap,C1.act,C1.rem
    FROM  Import_Courses2 AS C1
    LEFT JOIN PROFESSORS 
    ON C1.primary_instructor=PROFESSORS.name
    LEFT JOIN TERM_CATALOG USING (term)
    LEFT JOIN COURSE_CATALOG
    ON C1.catalog_id = COURSE_CATALOG.catalog_id AND COURSE_CATALOG.catalog_year=TERM_CATALOG.catalog_year
    ORDER BY crn;

 * sqlite:///CourseData.db
31874 rows affected.
31874 rows affected.


[]

In [47]:
%%sql
SELECT *
FROM COURSES AS C1 LIMIT 2;

 * sqlite:///CourseData.db
Done.


course_id,crn,catalog_id,term_id,section,professor_id,cap,act,rem
1,10893,BI 0075,31,1,Thomas B. Cunningham,20,22,-2
2,10893,BI 0075,32,1,Thomas B. Cunningham,20,22,-2


In [48]:
%%sql
SELECT *
FROM PROFESSORS AS C1 LIMIT 2;

 * sqlite:///CourseData.db
Done.


professor_id,name
1,Michael P. Coyne
2,Rebecca I. Bloch


## Erik 

When I find out how to save and build on querries I will be answering the following question. 
- Nbr of active students over the years 
- Most heavy hours per location and time 
- Each semester which professors have more students than capacity, same for department, subject 
- Vice versa which professors work too little 

## 1.
- need to update ERD 
- Courses table doesn’t have any key that would connect to the Professors table
- Professors_id are names of the professors. 
- So the only way to join courses and professors is to type ON C1.professor_id = PROFESSORS.name; 
- So professors_id in COURSES table is names and its numbers in PROFESSORS table. 

I think we should uniquely connect the two tables e.g. maybe crn/term_id and put that into PROFESSORS table as foreign key?

Because now we are storing names of professors in 2 tables I think that’s unnecessary (might as well drop the professors table then) 


In [50]:
%%sql
SELECT *
FROM COURSES AS C1
JOIN PROFESSORS 
ON C1.professor_id = PROFESSORS.name LIMIT ;

 * sqlite:///CourseData.db
Done.


course_id,crn,catalog_id,term_id,section,professor_id,cap,act,rem,professor_id_1,name
1,10893,BI 0075,31,1,Thomas B. Cunningham,20,22,-2,47,Thomas B. Cunningham
2,10893,BI 0075,32,1,Thomas B. Cunningham,20,22,-2,47,Thomas B. Cunningham


There are dates in professor_id for some reason. I think this was in our original files so its their mistake but we should still fix it I think. 

In [60]:
%%sql
SELECT DISTINCT professor_id, SUM(cap), SUM(act), SUM(rem)
FROM COURSES
GROUP BY (professor_id) LIMIT 10;

 * sqlite:///CourseData.db
Done.


professor_id,SUM(cap),SUM(act),SUM(rem)
01/20-04/30,0,0,0
01/20-05/08,0,0,0
05/21-08/17,2,-2,0
08/15-01/15,0,20,0
09/01-12/07,0,0,0
09/02-12/19,0,0,1998
09/02-12/30,2,0,0
09/15-10/31,0,0,0
Aaron K. Perkus,40,44,-4
Aaron Quinn Weinstein,400,252,148


### Historical Overcap professors 

In [84]:
%%sql
SELECT DISTINCT professor_id, cap, act, rem
FROM COURSES
WHERE rem < 0 AND cap > 0
GROUP BY professor_id LIMIT 2;

 * sqlite:///CourseData.db
Done.


professor_id,cap,act,rem
Aaron R. Van Dyke,15,21,-6
Adam E. Rugg,27,28,-1


### Last year overcap professors
- courses, course meetings

# 2. **course meethings table empty?**

In [163]:
%%sql
SELECT * FROM COURSE_MEETINGS;

 * sqlite:///CourseData.db
Done.


crn,location,day,start,end


### over/under capacaty courses 

In [156]:
%%sql
SELECT DISTINCT course_title, rem, prereqs, catalog_year
FROM COURSE_CATALOG CC
JOIN COURSES C1
USING (catalog_id)
WHERE rem < 0 AND cap >0 LIMIT 10;

 * sqlite:///CourseData.db
Done.


course_title,rem,prereqs,catalog_year
Ecology and Society,-2,None,2017_2018
Ecology and Society,-2,None,2018_2019
Watercolor,-3,None,2017_2018
Watercolor,-3,None,2018_2019
Legal Environment of Business,-1,Junior standing.,2017_2018
Legal Environment of Business,-1,Junior standing.,2018_2019
Ecology and Society,-1,None,2017_2018
Ecology and Society,-1,None,2018_2019
The Nature of the Universe,-1,None,2017_2018
The Nature of the Universe,-1,None,2018_2019


### Number of students over years
3 joins for that courses, course meetings, and course catalogs

In [ ]:
%%sql
SELECT DISTINCT course_title, rem, prereqs, catalog_year
FROM COURSE_CATALOG CC
JOIN COURSES C1
FROM 
USING (catalog_id)
WHERE rem < 0 AND cap >0 LIMIT 10;

### Are courses that have prereqs more overcapacity?

In [165]:
%%sql
SELECT DISTINCT course_title, cap, act, rem, prereqs
FROM COURSE_CATALOG CC
JOIN COURSES C1
USING (catalog_id)
WHERE rem < 0  AND prereqs != 'None' LIMIT 10;

 * sqlite:///CourseData.db
Done.


course_title,cap,act,rem,prereqs
Legal Environment of Business,0,21,-21,Junior standing.
Legal Environment of Business,0,23,-23,Junior standing.
Legal Environment of Business,0,24,-24,Junior standing.
Legal Environment of Business,0,25,-25,Junior standing.
Legal Environment of Business,0,26,-26,Junior standing.
Legal Environment of Business,0,27,-27,Junior standing.
Legal Environment of Business,0,28,-28,Junior standing.
Legal Environment of Business,0,30,-30,Junior standing.
Legal Environment of Business,15,16,-1,Junior standing.
Legal Environment of Business,20,21,-1,Junior standing.


### PROGRAM table works

In [166]:
%%sql
SELECT * FROM PROGRAMS;

 * sqlite:///CourseData.db
Done.


program_id,program_code,program_name
1,AC,Accounting
2,AE,Applied Ethics
3,AH,Art History
4,AN,Asian Studies
5,AR,Arabic
6,AS,American Studies
7,AY,Anthropology
8,BB,Business
9,BEN,Bioengineering
10,BI,Biology
